# Answering Business Questions using SQL

In this guided project, we're going to practice using our SQL skills to answer business questions.

Firstly, we will explore the database a little, to get more used to it.

In [4]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db
    
#conn = sqlite3.connect("chinook.db")
#cursor = conn.cursor()

In [5]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Selecting Albums to Purchase 
The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

Artist Name | Genre
--- | --- 
Regal | Hip-hop
Red Tone | Punk
Meteor and the Girls | Pop
Slim Jim Bites | Blues

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

We'll need to write a query to find out which genres sell the most tracks in the USA, write up a summary of your findings, and make a recommendation for the three artists whose albums we should purchase for the store.

In [6]:
%%sql
WITH
    usa_tracks_sold AS
    (
    SELECT
        il.*
    FROM invoice_line il
    INNER JOIN invoice i ON i.invoice_id = il.invoice_id
    INNER JOIN customer c ON c.customer_id = i.customer_id
    WHERE c.country = 'USA'
    ),
    track_genre AS
    (
    SELECT
        t.track_id track_id,
        t.name track_name,
        g.name genre
    FROM track t
    INNER JOIN genre g ON g.genre_id = t.genre_id
    )

SELECT 
    tg.genre genre,
    COUNT(uts.invoice_line_id) tracks_sold,
    ROUND(
            CAST(COUNT(uts.invoice_line_id) as Float)/
                                    (SELECT COUNT(*)
                                     FROM usa_tracks_sold
                                    ), 4
         ) tracks_sold_percentage
FROM usa_tracks_sold uts
INNER JOIN track_genre tg ON tg.track_id = uts.track_id
GROUP BY 1
ORDER BY 2 DESC;

 * sqlite:///chinook.db
Done.


genre,tracks_sold,tracks_sold_percentage
Rock,561,0.5338
Alternative & Punk,130,0.1237
Metal,124,0.118
R&B/Soul,53,0.0504
Blues,36,0.0343
Alternative,35,0.0333
Latin,22,0.0209
Pop,22,0.0209
Hip Hop/Rap,20,0.019
Jazz,14,0.0133


As we can only choose three of the four albums shown, based on the number of sales per genre, we can assume that the most appropriate ones to be added are from the artists:
- Red Tone (Punk)
- Slim Jim Bites (Blues)
- Meteor and the Girls (Pop)

It's worth noticing that the vast majority of tracks sold in the USA are Rock ones with 53.38% of all sales, so we should be on the lookout for artists and albums from the 'rock' genre.

## Analyzing Employee Sales Performance

We are going to find the total dollar amount of sales assigned to each sales support agent within the company to see if any one of them is performing either better or worse than the others.

In [8]:
%%sql
WITH
    sales_support_agent AS
    (  
    SELECT
        employee_id agent_id,
        e.first_name || " " || e.last_name agent_name,
        hire_date        
    FROM employee e
    WHERE title = 'Sales Support Agent'
    ),
    customers_buys AS
    (
    SELECT
        i.customer_id customer_id,
        COUNT(i.invoice_id) total_buys,
        SUM(i.total) total_spent,
        c.support_rep_id agent_id        
    FROM invoice i
    INNER JOIN customer c ON c.customer_id = i.customer_id
    GROUP BY 1
    )
    
SELECT
    ssa.agent_id agent_id,
    ssa.agent_name agent_name,
    ssa.hire_date hire_date,
    SUM(cb.total_buys) total_sales,
    COUNT(cb.customer_id) total_customers,
    SUM(cb.total_spent) total_dollars_sold,
    CAST(SUM(cb.total_spent) as Float)/
    CAST(SUM(cb.total_buys) as Float) dollars_per_sale
FROM sales_support_agent ssa
INNER JOIN customers_buys cb ON cb.agent_id = ssa.agent_id
GROUP BY 1
ORDER BY 5 DESC

 * sqlite:///chinook.db
Done.


agent_id,agent_name,hire_date,total_sales,total_customers,total_dollars_sold,dollars_per_sale
3,Jane Peacock,2017-04-01 00:00:00,212,21,1731.5099999999998,8.167499999999999
4,Margaret Park,2017-05-03 00:00:00,214,20,1584.0000000000002,7.4018691588785055
5,Steve Johnson,2017-10-17 00:00:00,188,18,1393.92,7.4144680851063836


We can see that there is a 20% difference between the top sales support agent and the one that sold less, but this is explained by the different hire dates. The employees have similar dollars per sale, although Jane Peacock has a bit better score than the others.

## Analyzing Sales by Country

In this task, we'll analyze the sales data for customers from each different country. In particular, we want to find out, for each country, the total number of customers, the total value of sales, the average value of sales per customer and the average order value.

Because there are a number of countries with only one customer, we will group these customers as "Other" in the analysis. 

In [12]:
%%sql

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / count(distinct customer_id) customer_lifetime_value,
        SUM(unit_price) / count(distinct invoice_id) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

 * sqlite:///chinook.db
Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056


We can observe that there may be good opportunities in Czech Republic, United Kingdom and India, since orders from these places are, on average, bigger than other countries. It's worth keeping in mind, though, that these countries has very few customers, so small marketing campaigns would be more suited.

## Albums vs Individual Tracks

In [11]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

 * sqlite:///chinook.db
Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


Album purchases account for almost 19% of purchases. Based on this data, I would not recommend purchasing only select tracks from albums from record companies, since there is potential of losing one fifth of revenue.

